# **Week 1 · Lesson 2 — Starter Notebook: Static Word Embeddings**


In this notebook you will build two flavours of 200‑dimensional word vectors on a tiny slice of Wikipedia (≈ 10 MB).

1) Count‑based:  PPMI  →  Truncated SVD

2) Prediction‑based:  Skip‑Gram with negative sampling

Fill every section marked `## TODO` and run all cells before submission.

**SECTION 0 ─ Imports & Dataset**

In [2]:
!pip -q install nltk gensim scikit-learn tqdm --upgrade

import nltk, re, json, math, random, itertools
from collections import Counter, defaultdict
from pathlib import Path
from tqdm import tqdm
import numpy as np
from sklearn.decomposition import TruncatedSVD
from gensim.models import Word2Vec

nltk.download('punkt')

# Download a miniature Wiki dump (≈ 10 MB plain text).
!wget -q https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O tiny_wiki.txt

RAW_PATH = Path('tiny_wiki.txt')
print("Raw corpus size:", RAW_PATH.stat().st_size / 1e6, "MB")

# from datasets import load_dataset
# raw_ds = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
# TEXT   = "\n".join(raw_ds["text"])
# print(TEXT)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Raw corpus size: 1.115394 MB


**SECTION 1 ─ Tokenisation helper (✅ implemented for you)**

In [3]:
TOKEN_RE = re.compile(r"[A-Za-z']+")

def sent_tokenise(text: str):
    for sent in nltk.sent_tokenize(text):
        yield [tok.lower() for tok in TOKEN_RE.findall(sent)]

nltk.download('punkt_tab')

sentences = list(sent_tokenise(RAW_PATH.read_text(encoding='utf-8')))
print("Example sentence tokens:", sentences[0][:12])

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Example sentence tokens: ['first', 'citizen', 'before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak']


**SECTION 2 ─ Build 5k vocabulary ** (## TODO)

Fill: build a frequency counter, keep the 5 000 most common tokens, map them to indices.  Use index 0 for <UNK>.

In [4]:
VOCAB_SIZE = 5000
UNK = "<unk>"

# ## TODO ---------------------------------------------------------------------
word_freq = Counter()
for sent in sentences:
    word_freq.update(sent)

most_common = [w for w, _ in word_freq.most_common(VOCAB_SIZE - 1)]
idx2word = [UNK] + most_common
word2idx = {w: i for i, w in enumerate(idx2word)}
print("Vocab built, size:", len(idx2word))


Vocab built, size: 5000


**SECTION 3 ─ ±5 window co‑occurrence counts**

In [5]:
WINDOW = 5
cooc = defaultdict(Counter)

# ## TODO ---------------------------------------------------------------------
for sent in sentences:
    indices = [word2idx.get(w, 0) for w in sent]
    for i, center in enumerate(indices):
        start = max(0, i - WINDOW)
        end   = min(len(indices), i + WINDOW + 1)
        for j in range(start, end):
            if i == j: continue
            context = indices[j]
            cooc[center][context] += 1
print("Center vocab with at least one context:", len(cooc))

Center vocab with at least one context: 4999


**SECTION 4 ─ PPMI matrix  (## TODO) **

In [6]:
print("Computing PPMI… (this may take 1‑2 min)")
# Precompute totals
sum_all = sum(sum(ct.values()) for ct in cooc.values())
row_tot = np.zeros(VOCAB_SIZE)
col_tot = np.zeros(VOCAB_SIZE)
for i, ctxs in cooc.items():
    row_tot[i] = sum(ctxs.values())
    for j, c in ctxs.items():
        col_tot[j] += c

# Build sparse tuples (i, j, ppmi)
rows, cols, vals = [], [], []
for i, ctxs in cooc.items():
    for j, c in ctxs.items():
        p_ij = c / sum_all
        p_i  = row_tot[i] / sum_all
        p_j  = col_tot[j] / sum_all
        ppmi = max(0.0, math.log2(p_ij / (p_i * p_j)))
        if ppmi > 0:
            rows.append(i); cols.append(j); vals.append(ppmi)
print("PPMI non‑zeros:", len(vals))

# Build sparse ←→ dense matrix (dense for SVD on 5k x 5k is OK)
PPMI = np.zeros((VOCAB_SIZE, VOCAB_SIZE), dtype=np.float32)
PPMI[rows, cols] = vals

Computing PPMI… (this may take 1‑2 min)
PPMI non‑zeros: 455548


**SECTION 5 ─ Truncated SVD (200‑d)  (## TODO)**

In [7]:
print("Running SVD…")
SVD_DIM = 200
svd = TruncatedSVD(n_components=SVD_DIM, random_state=42)
count_vecs = svd.fit_transform(PPMI)
print("Count‑based vectors shape:", count_vecs.shape)


Running SVD…
Count‑based vectors shape: (5000, 200)


**SECTION 6 ─ Skip‑Gram training (gensim)  (✅ code provided)**

In [8]:
print("Training Skip‑Gram model…")
sg_sentences = [[w for w in sent if w in word2idx] for sent in sentences]
sg_model = Word2Vec(sg_sentences, vector_size=200, window=5, min_count=1,
                    sg=1, negative=5, epochs=5, workers=4, seed=42)
sg_vecs = np.zeros_like(count_vecs)
for w, i in word2idx.items():
    sg_vecs[i] = sg_model.wv[w] if w in sg_model.wv else np.zeros(SVD_DIM)
print("Skip‑Gram vectors ready.")


Training Skip‑Gram model…
Skip‑Gram vectors ready.


**SECTION 7 ─ Helper: nearest neighbours + analogy**

In [11]:
def nn(word: str, vecs: np.ndarray, topk: int = 10):
    """Return nearest-neighbour tokens by cosine similarity."""
    if word not in word2idx:                    # word unseen in vocab
        return ["<unk>"]
    idx = word2idx[word]
    w_vec = vecs[idx]
    if not w_vec.any():                        # zero vector -> no info
        return ["<zero-vector>"]
    # cosine similarity (eps avoids /0)
    sims = vecs @ w_vec / (
        np.linalg.norm(vecs, axis=1) * np.linalg.norm(w_vec) + 1e-9
    )
    best = sims.argsort()[::-1]                # descending
    out  = [idx2word[i] for i in best if i != idx][:topk]
    return out or ["<none>"]

def analogy(a, b, c, vecs):
    """Return d such that b-a+c ≈ d."""
    for w in (a, b, c):
        if w not in word2idx or not vecs[word2idx[w]].any():
            return "<missing-vector>"
    vec = vecs[word2idx[b]] - vecs[word2idx[a]] + vecs[word2idx[c]]
    sims = vecs @ vec / (
        np.linalg.norm(vecs, axis=1) * np.linalg.norm(vec) + 1e-9
    )
    for idx in sims.argsort()[::-1]:
        if idx not in (word2idx[a], word2idx[b], word2idx[c]):
            return idx2word[idx]
    return "<none>"

**SECTION 8 ─ Evaluation  (## TODO)**

In [12]:
print("Nearest neighbours for cat, economy, happy\n")
for name, vecs in [("PPMI–SVD", count_vecs), ("Skip‑Gram", sg_vecs)]:
    print(name, "— cat →", nn("cat", vecs))
    print(name, "— economy →", nn("economy", vecs))
    print(name, "— happy →", nn("happy", vecs))

print("\nAnalogy king - man + woman → ?\n")
for name, vecs in [("PPMI–SVD", count_vecs), ("Skip‑Gram", sg_vecs)]:
    print(name, "→", analogy("man", "king", "woman", vecs))


Nearest neighbours for cat, economy, happy

PPMI–SVD — cat → ['mouse', "shunn'd", 'dog', 'scratch', 'budge', 'every', "ne'er", 'unworthy', "man's", 'little']
PPMI–SVD — economy → ['<unk>']
PPMI–SVD — happy → ['befal', 'days', 'joyful', 'watchful', 'doubt', 'victory', 'blessings', 'girl', "'mongst", 'enrich']
Skip‑Gram — cat → ['toys', 'non', 'foreign', 'fold', 'silk', 'wisest', 'wither', "kings'", 'criminal', 'din']
Skip‑Gram — economy → ['<unk>']
Skip‑Gram — happy → ["say'st", 'tall', 'prospero', 'fond', 'coward', 'widow', "know'st", 'tut', 'comest', 'woman']

Analogy king - man + woman → ?

PPMI–SVD → richard
Skip‑Gram → bolingbroke
